# Data mining 1st Assignment
<b>Ευστράτιος Ζωγραφάκης 1115201600049

Κωνσταντίνος Σκορδούλης 1115201600155 </b>

 First of all, we will try to fix the missing data problem for each month. 
 Then we will merge the files, creating our train.csv

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
listings_cols = ['id','zipcode','transit','bedrooms','beds','review_scores_rating','number_of_reviews','neighbourhood','name','latitude','longitude','last_review','instant_bookable','host_since','host_response_rate','host_identity_verified','host_has_profile_pic','first_review','description','city','cancellation_policy','bed_type','bathrooms','accommodates','amenities','room_type','property_type','price','availability_365','minimum_nights']
listings_cols0 = ['id','name','neighbourhood','latitude','longitude','room_type','price','minimum_nights','number_of_reviews','last_review','availability_365']



Create the dataframe for each file

In [3]:
######################### FEBRUARY ########################################

flist_path = r'./data/febrouary/listings.csv'
flist_df = pd.read_csv(flist_path, usecols=listings_cols)

flist_path0 = r'./data/febrouary/listings0.csv'
flist_df0 = pd.read_csv(flist_path0, usecols=listings_cols0)

fcal_path = r'./data/febrouary/calendar.csv'
fcal_df = pd.read_csv(fcal_path)

frev_path = r'./data/febrouary/reviews.csv'
frev_df = pd.read_csv(frev_path)


################## MARCH ###########################################

mlist_path = r'./data/march/listings.csv'
mlist_df = pd.read_csv(mlist_path, usecols=listings_cols)

mlist_path0 = r'./data/march/listings0.csv'
mlist_df0 = pd.read_csv(mlist_path0, usecols=listings_cols0)

mcal_path = r'./data/march/calendar.csv'
mcal_df = pd.read_csv(mcal_path)

mrev_path = r'./data/march/reviews.csv'
mrev_df = pd.read_csv(mrev_path)


################## APRIL ###########################################

alist_path = r'./data/april/listings.csv'
alist_df = pd.read_csv(alist_path, usecols=listings_cols)

alist_path0 = r'./data/april/listings0.csv'
alist_df0 = pd.read_csv(alist_path0, usecols=listings_cols0)

acal_path = r'./data/april/calendar.csv'
acal_df = pd.read_csv(acal_path)

arev_path = r'./data/april/reviews.csv'
arev_df = pd.read_csv(arev_path)

Looking through the files, we have gathered that some fields can be "retrieved" from the other files:

( R -> Reviews.csv, 0 ->listings0.csv, C -> calendar.csv)

1) Number_of_reviews ==> R,0

2) Neighbourhood ==> 0 
 (because in the "neighbourhood" column of listing.csv there is a lot of trash, we will get the values from listing0.csv)

3) Name ==> 0

4) Latitude ==> 0

5) Longtitude ==> 0

6) Last_review ==> R,0

7) First_review ==> R

8) Room_type ==> 0

9) Price ==> C,0

10) Availability_365 ==> 0

11) Minimum_nights ==> C,0

In [4]:
#check_field = ['name','neigbourhood','latitude','longtitude','room_type','price','minimum_nights','number_of_reviews','last_review','availability_365']

check_field = ['number_of_reviews','neighbourhood','name','latitude','longitude','last_review','first_review','room_type','price','availability_365','minimum_nights']

#check_field0 = ['name','neighbourhood','latitude','longitude','room_type','price','minimum_nights','number_of_reviews','last_review','availability_365']

For each row check if the any of the check_fields is NaN. If yes, try to find a new valid value from the rest of the files.

If the rest of the files have NaN ==> do nothing

In [5]:
def getIndex(df,id_input,file):
    
    if file == '0': #listings0.csv , listings.csv
        result = df.id[ df.id == id_input].index.tolist()
        
        if len(result) != 1:
            sys.exit("expected length = 1")
        
    elif (file == 'R') or (file == 'C'): #caldendar.csv or reviews.csv
        result = df.listing_id[ df.listing_id == id_input].index.tolist()
    else:
        result = None
    return result      # returns a list of the indexes ( which have the same id) or None    

We have noticed that some columns of listing, have values but they are ??? values. 
Specifically, neighbouhrhood and room_type has some numeric values?? ( there maybe more columns like that).

Listings0.csv however has the correct values. As such, we re-assign the values of listings0 to listings ( except if listings0 has NaN).

In [6]:
def merge(df,df0,f,id_input,i): 
    # f is the field , which we overwrite
    
    index = getIndex(df0,id_input,'0')[0]
    
    #if df0.at[index,f] is not None:
    if pd.Series([df0.at[index,f]]).isna().item() == False:
        df.at[i,f] = df0.at[index,f]
        
        return [True,df] #assignment was done
        
    #else do nothing    
    return [False,df]   #nothing happened

However listings0 does not have all our columns, so we draw out info from the other files.

Each of the functions below try to fix the "missing data" by searching the other files

In [7]:
def Fix_numberOfReviews(list_df,rev_df,i,list_id):
    #i is the index/row of listings.csv
    
    f = 'number_of_reviews'
                      
    #we check Reviews.csv 
    temp_index_list = getIndex(rev_df,list_id,'R')
    list_df.at[i,f] = len(temp_index_list)
    
    return list_df

In [8]:
def Fix_firstReview(list_df,rev_df,i,list_id):
    
    f = 'first_review'
                      
    #we check Reviews.csv
    
    #x = revrev_df.date[ rev_df.listing_id == list_id ].head(1)
    #list_df.at[i,f] = x.item()
    
    list_df.at[i,f] = rev_df.date[ rev_df.listing_id == list_id ].head(1).item() #find the oldest_review  
    
    return list_df
    

In [9]:
def Fix_lastReview(list_df,rev_df,i,list_id):
    
    f = 'last_review'
                      
    #we check Reviews.csv
    x = rev_df.date[ rev_df.listing_id == list_id ].sort_values(ascending=False).head(1)
    #print("len of x is %d" %(len(x)) )
    #print("list id is %d" %(list_id))
    
    if( len(x) == 0): #didn't find a review
        return list_df #just return
    
    list_df.at[i,f] = x.item()
    
    #list_df.at[i,f] = rev_df.date[ rev_df.listing_id == list_id ].sort_values(ascending=False).head(1).item() #find the newest_review  
    
    return list_df
    

In [10]:
def Fix_price(list_df,cal_df,i,list_id):
    
    f = 'price'
    
    price = cal_df.price[ cal_df.listing_id == list_id ].replace('[\$,]', '', regex=True).astype(float).mean()
    price = '$' + str(round(price)) + '.00'
    
    #we check cal_df
    list_df.at[i,f] = price
    
    return list_df    

In [11]:
def Fix_minimumNights(list_df,cal_df,i,list_id):
    
    f = 'minimum_nights'
    
    minimum_nights = cal_df.minimum_nights[ cal_df.listing_id == list_id ].head(1).item()
    
    #we check cal_df
    list_df.at[i,f] = minimum_nights
    
    return list_df    

# February

In [12]:
for i in flist_df.index:
    
    list_id = flist_df.at[i,'id']
    
    for f in check_field:
        
        if f == 'first_review':
            
            if pd.Series([flist_df.at[i,f]]).isna().item() == False: #do nothing
                continue
        
            #else find first_review from review.csv            
            flist_df = Fix_firstReview(flist_df,fcal_df,i,list_id)
            continue
            #finished 
            
        ###############else ==> merge() with listing0 field ######################
        
        #df,df0,f,id_input,i
        
        result = merge(flist_df,flist_df0,f,list_id,i)        
        if result[0] == True:
            flist_df = result[1]
            continue
        
        #else listing0 had None ==> search other files
        
        #only check fields that are NaN
        if pd.Series(flist_df.at[i,f]).isna().item() == False: 
            continue
        
        #checking which field
        if f == 'number_of_reviews':
            flist_df = Fix_numberOfReviews(flist_df,frev_df,i,list_id)
            continue
        elif f == 'last_review':
            flist_df = Fix_lastReview(flist_df,frev_df,i,list_id)
            continue
        elif f == 'price':
            flist_df = Fix_price(flist_df,fcal_df,i,list_id)
            continue        
        elif f == 'minimum_nights':
            flist_df = Fix_minimumNights(flist_df,fcal_df,i,list_id)
            continue
            
        
#end for loop

#Add one final column 'month'
flist_df['month'] = 1

flist_df.to_csv(r'./data/febrouary/train_febrouary.csv') #finished
flist_df

,id,name,description,transit,host_since,host_response_rate,host_has_profile_pic,host_identity_verified,neighbourhood,city,...,price,minimum_nights,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,instant_bookable,cancellation_policy,month
0,10595,"96m2, 3BR, 2BA, Metro, WI-FI etc...",Athens Furnished Apartment No6 is 3-bedroom ap...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,71,1,294,17,2011-05-20,2019-01-12,96.0,t,strict_14_with_grace_period,1
1,10988,"75m2, 2-br, metro, wi-fi, cable TV",Athens Furnished Apartment No4 is 2-bedroom ap...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,82,1,0,31,2012-10-21,2017-11-23,92.0,t,strict_14_with_grace_period,1
2,10990,"50m2, Metro, WI-FI, cableTV, more",Athens Furnished Apartment No3 is 1-bedroom ap...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,47,1,282,27,2012-09-06,2019-02-01,97.0,t,strict_14_with_grace_period,1
3,10993,"Studio, metro, cable tv, wi-fi, etc",The Studio is an -excellent located -close t...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,37,1,286,42,2012-09-24,2019-02-02,97.0,t,strict_14_with_grace_period,1
4,10995,"47m2, close to metro,cable TV,wi-fi",AQA No2 is 1-bedroom apartment (47m2) -excell...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,47,2,308,16,2010-07-08,2019-01-11,95.0,t,strict_14_with_grace_period,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9095,32226813,Zoe's Studio with Balcony Nefeli,"To Zoe´s Studio είναι ιδανικό για ζευγάρια, ή ...","Δίπλα στο σπίτι θα βρείτε 2 στάσεις μετρό, αυτ...",2010-05-26,100%,t,t,ΝΕΟΣ ΚΟΣΜΟΣ,Athens,...,40,2,353,0,2019-02-08,NaN,NaN,t,moderate,1
9096,32227307,"Cozy and modern studio, next to metro station",Very comfortable and cozy on the 2nd floor wit...,about 300m walking distance metro station meta...,2017-06-28,NaN,t,f,ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ,Athina,...,26,1,230,0,2019-02-08,NaN,NaN,t,flexible,1
9097,32228418,Venia luxury suite,"Compositions: 1 Bedroom, 1 Bathroom, 1 Kitchen...",NaN,2017-06-17,100%,t,f,ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,Athina,...,126,1,357,0,2019-02-08,NaN,NaN,t,flexible,1
9098,32228956,Ανακαινισμένο διαμέρισμα αυτόνομο !!!!,Εχει γίνει γενική ανακαινιση σε όλους τους χώρ...,"τρόλεϊ , ,ταξί ,και είναι δέκα λεπτά απο την...",2018-04-13,NaN,t,f,ΚΥΨΕΛΗ,Athina,...,45,1,160,0,2019-02-08,NaN,NaN,t,flexible,1


# March

In [13]:
for i in mlist_df.index:
    
    list_id = mlist_df.at[i,'id']
    
    for f in check_field:
        
        if f == 'first_review':
            
            if pd.Series(mlist_df.at[i,f]).isna().item() == False: #do nothing
                continue
        
            #else find first_review from review.csv
            mlist_df = Fix_firstReview(mlist_df,mcal_df,i,list_id)
            continue
            #finished 
            
        ###############else ==> merge() with listing0 field ######################
        
        #df,df0,f,id_input,i
        
        result = merge(mlist_df,mlist_df0,f,list_id,i)        
        if result[0] == True:
            mlist_df = result[1]
            continue
        
        #else listing0 had None ==> search other files
        
        #only check fields that are NaN
        if pd.Series(mlist_df.at[i,f]).isna().item() == False: 
            continue
        
        #checking which field
        if f == 'number_of_reviews':
            mlist_df = Fix_numberOfReviews(mlist_df,mrev_df,i,list_id)
            continue
        elif f == 'last_review':
            mlist_df = Fix_lastReview(mlist_df,mrev_df,i,list_id)
            continue
        elif f == 'price':
            mlist_df = Fix_price(mlist_df,mcal_df,i,list_id)
            continue        
        elif f == 'minimum_nights':
            mlist_df = Fix_minimumNights(mlist_df,mcal_df,i,list_id)
            continue
            
        
#end for loop

#Add one final column 'month'
mlist_df['month'] = 2

mlist_df.to_csv(r'./data/march/train_march.csv') #finished
mlist_df

,id,name,description,transit,host_since,host_response_rate,host_has_profile_pic,host_identity_verified,neighbourhood,city,...,price,minimum_nights,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,instant_bookable,cancellation_policy,month
0,10595,"96m2, 3BR, 2BA, Metro, WI-FI etc...",Athens Furnished Apartment No6 is 3-bedroom ap...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,72,1,262,17,2011-05-20,2019-01-12,96.0,t,strict_14_with_grace_period,2
1,10988,"75m2, 2-br, metro, wi-fi, cable TV",Athens Furnished Apartment No4 is 2-bedroom ap...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,82,1,0,31,2012-10-21,2017-11-23,92.0,t,strict_14_with_grace_period,2
2,10990,"50m2, Metro, WI-FI, cableTV, more",Athens Furnished Apartment No3 is 1-bedroom ap...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,48,1,255,29,2012-09-06,2019-03-03,97.0,t,strict_14_with_grace_period,2
3,10993,"Studio, metro, cable tv, wi-fi, etc",The Studio is an -excellent located -close t...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,38,1,246,44,2012-09-24,2019-03-03,96.0,t,strict_14_with_grace_period,2
4,10995,"47m2, close to metro,cable TV,wi-fi",AQA No2 is 1-bedroom apartment (47m2) -excell...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,48,2,280,16,2010-07-08,2019-01-11,95.0,t,strict_14_with_grace_period,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9356,32956579,Flat for 3 persons 10 minutes for center of At...,Το διαμέρισμα βρίσκεται σε μία ήσυχη γειτονιά ...,NaN,2019-03-04,NaN,t,f,ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ,Athina,...,30,1,173,0,2019-03-11,NaN,NaN,t,flexible,2
9357,32958153,TRADITIONAL VINTAGE ROOM IN ATHENS 2,A unique traditional style house close to the ...,There is a bus stop two min from the house (3 ...,2019-03-10,NaN,t,f,ΙΛΙΣΙΑ,Zografou,...,14,1,344,0,2019-03-11,NaN,NaN,t,flexible,2
9358,32958286,TRADITIONAL VINTAGE ROOM 3,A unique traditional style house close to the ...,There is a bus stop two min from the house (3 ...,2019-03-10,NaN,t,f,ΙΛΙΣΙΑ,Zografou,...,14,1,344,0,2019-03-11,NaN,NaN,t,flexible,2
9359,32958368,TRADITIONAL SINGLE ROOM WITH PRIVET BALCONY,A unique traditional style house close to the ...,There is a bus stop two min from the house (3 ...,2019-03-10,NaN,t,f,ΙΛΙΣΙΑ,Zografou,...,12,1,344,0,2019-03-11,NaN,NaN,t,flexible,2


# April

In [14]:
for i in alist_df.index:
    
    list_id = alist_df.at[i,'id']
    
    for f in check_field:
        
        if f == 'first_review':
            
            if pd.Series(alist_df.at[i,f]).isna().item() == False: #do nothing
                continue
        
            #else find first_review from review.csv
            alist_df = Fix_firstReview(alist_df,acal_df,i,list_id)
            continue
            #finished 
            
        ###############else ==> merge() with listing0 field ######################
        
        #df,df0,f,id_input,i
        
        result = merge(alist_df,alist_df0,f,list_id,i)        
        if result[0] == True:
            alist_df = result[1]
            continue
        
        #else listing0 had None ==> search other files
        
        #only check fields that are NaN
        if pd.Series(alist_df.at[i,f]).isna().item() == False: 
            continue
        
        #checking which field
        if f == 'number_of_reviews':
            alist_df = Fix_numberOfReviews(alist_df,arev_df,i,list_id)
            continue
        elif f == 'last_review':
            alist_df = Fix_lastReview(alist_df,arev_df,i,list_id)
            continue
        elif f == 'price':
            alist_df = Fix_price(alist_df,acal_df,i,list_id)
            continue        
        elif f == 'minimum_nights':
            alist_df = Fix_minimumNights(alist_df,acal_df,i,list_id)
            continue
            
        
#end for loop

#Add one final column 'month'
alist_df['month'] = 3

alist_df.to_csv(r'./data/april/train_april.csv') #finished
alist_df

,id,name,description,transit,host_since,host_response_rate,host_has_profile_pic,host_identity_verified,neighbourhood,city,...,price,minimum_nights,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,instant_bookable,cancellation_policy,month
0,10595,"96m2, 3BR, 2BA, Metro, WI-FI etc...",Athens Furnished Apartment No6 is 3-bedroom ap...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,73,1,222,19,2011-05-20,2019-04-08,97.0,t,strict_14_with_grace_period,3
1,10988,"75m2, 2-br, metro, wi-fi, cable TV",Athens Furnished Apartment No4 is 2-bedroom ap...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,82,1,0,31,2012-10-21,2017-11-23,92.0,t,strict_14_with_grace_period,3
2,10990,"50m2, Metro, WI-FI, cableTV, more",Athens Furnished Apartment No3 is 1-bedroom ap...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,49,1,234,29,2012-09-06,2019-03-03,97.0,t,strict_14_with_grace_period,3
3,10993,"Studio, metro, cable tv, wi-fi, etc",The Studio is an -excellent located -close t...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,38,1,226,45,2012-09-24,2019-04-04,96.0,t,strict_14_with_grace_period,3
4,10995,"47m2, close to metro,cable TV,wi-fi",AQA No2 is 1-bedroom apartment (47m2) -excell...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,49,2,249,17,2010-07-08,2019-04-04,95.0,t,strict_14_with_grace_period,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9656,33819692,Lysicrates Monument - Athens 33,"""Athens 33"" is a newly renovated building in N...",NaN,2014-05-09,100%,t,t,ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,Athina,...,90,2,260,0,2019-04-13,NaN,NaN,t,flexible,3
9657,33820145,Bright and Stylish Exarchia apartment,"Newly renovated, this two bedroom apartment is...",Omonia metro station is 600 m away from the ap...,2019-04-12,NaN,t,f,ΜΟΥΣΕΙΟ-ΕΞΑΡΧΕΙΑ-ΝΕΑΠΟΛΗ,Athina,...,50,2,263,0,2019-04-13,NaN,NaN,t,moderate,3
9658,33821757,Athens21,Athens 21 is centrally located right next to M...,NaN,2019-04-12,NaN,t,f,ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,Athina,...,200,1,86,0,2019-04-13,NaN,NaN,t,flexible,3
9659,33821833,Tower of Winds,"""Athens 33"" is a newly renovated building in N...",NaN,2014-05-09,100%,t,t,ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,Athens,...,90,2,260,0,2019-04-13,NaN,NaN,t,flexible,3


# Finale

Now we just need to concat the 3 files ===> train.csv

In [15]:
#train_df = pd.concat([flist_df,mlist_df,alist_df],ignore_index=True).reset_index(drop=True)

train_df = pd.concat([flist_df,mlist_df,alist_df]).reset_index(drop=True)
train_df.to_csv(r'./train.csv',index=False)
train_df

,id,name,description,transit,host_since,host_response_rate,host_has_profile_pic,host_identity_verified,neighbourhood,city,...,price,minimum_nights,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,instant_bookable,cancellation_policy,month
0,10595,"96m2, 3BR, 2BA, Metro, WI-FI etc...",Athens Furnished Apartment No6 is 3-bedroom ap...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,71,1,294,17,2011-05-20,2019-01-12,96.0,t,strict_14_with_grace_period,1
1,10988,"75m2, 2-br, metro, wi-fi, cable TV",Athens Furnished Apartment No4 is 2-bedroom ap...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,82,1,0,31,2012-10-21,2017-11-23,92.0,t,strict_14_with_grace_period,1
2,10990,"50m2, Metro, WI-FI, cableTV, more",Athens Furnished Apartment No3 is 1-bedroom ap...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,47,1,282,27,2012-09-06,2019-02-01,97.0,t,strict_14_with_grace_period,1
3,10993,"Studio, metro, cable tv, wi-fi, etc",The Studio is an -excellent located -close t...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,37,1,286,42,2012-09-24,2019-02-02,97.0,t,strict_14_with_grace_period,1
4,10995,"47m2, close to metro,cable TV,wi-fi",AQA No2 is 1-bedroom apartment (47m2) -excell...,Note: 5-day ticket for all the public transpor...,2009-09-08,100%,t,t,ΑΜΠΕΛΟΚΗΠΟΙ,Athens,...,47,2,308,16,2010-07-08,2019-01-11,95.0,t,strict_14_with_grace_period,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28117,33819692,Lysicrates Monument - Athens 33,"""Athens 33"" is a newly renovated building in N...",NaN,2014-05-09,100%,t,t,ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,Athina,...,90,2,260,0,2019-04-13,NaN,NaN,t,flexible,3
28118,33820145,Bright and Stylish Exarchia apartment,"Newly renovated, this two bedroom apartment is...",Omonia metro station is 600 m away from the ap...,2019-04-12,NaN,t,f,ΜΟΥΣΕΙΟ-ΕΞΑΡΧΕΙΑ-ΝΕΑΠΟΛΗ,Athina,...,50,2,263,0,2019-04-13,NaN,NaN,t,moderate,3
28119,33821757,Athens21,Athens 21 is centrally located right next to M...,NaN,2019-04-12,NaN,t,f,ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,Athina,...,200,1,86,0,2019-04-13,NaN,NaN,t,flexible,3
28120,33821833,Tower of Winds,"""Athens 33"" is a newly renovated building in N...",NaN,2014-05-09,100%,t,t,ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,Athens,...,90,2,260,0,2019-04-13,NaN,NaN,t,flexible,3


# END